In [311]:
import requests
from bs4 import BeautifulSoup

class RequestCustom:
    def __init__(self, url, site_name):
        self.url = url
        self.site_name = site_name

    def get_url(self):
        return self.url

    def get_request(self):
        url=self.get_url()     
        return requests.get(url, allow_redirects=False)

    def get_status(self):
        request = self.get_request()
        status = request.status_code
        return status

    def get_soup(self):
        request=self.get_request()
        status = self.get_status()
        if status == 200:
            soup = BeautifulSoup(request.text, 'html.parser')
            return soup
        else:
            print("Error: ", status)
            return None

In [312]:
import csv

class CsvCustom:

    def __init__(self, site_name):
        self.site_name = site_name

    def create_csv(self):
        file = open(f'{self.site_name}.csv', 'w',newline='',encoding='utf-8-sig')
        writer = csv.writer(file)
        headers = ["title", "link", "description", "price"]
        writer.writerow(headers)

    def write_csv(self,**kwargs): 
        title = kwargs['title']
        link = kwargs['link']
        description = kwargs['description']
        price = kwargs['price']

        file = open(f'{self.site_name}.csv', 'a', newline='', encoding='utf-8-sig')
        writer = csv.writer(file)
        contents = ([title,link, description, price])
        writer.writerow(contents)
        return file

    def close_csv(self,file):
        file.close()

In [313]:
listes_uri=[
    'Side-by-Side-Refrigerators.html',
    'bottom-freezer-refrigerator.html',
    'Side-by-Side-Refrigerators.html',
    'Top-Freezer-Refrigerators.html',
    'counter-depth-refrigerator.html',    
]
site_refrigerator_pro = RequestCustom(url=f"https://www.refrigeratorpro.com/{listes_uri[0]}", site_name="refrigerator_pro")

soup = site_refrigerator_pro.get_soup()

listes_prods_title=soup.find("ol").find_all("h3")
listes_prods_link=soup.find("ol").find_all("a")
listes_prods_description=soup.find("ol").find_all("p")

In [314]:
company_dict={}

for title in listes_prods_title:
    title = title.get_text().strip()
    for link in listes_prods_link:
                if title == "General Electric":
                    title = "GE_"
                if title.upper() in link["href"].strip().replace("-"," ").upper():                    
                    if title not in company_dict.keys():
                        company_dict[title]=link["href"]
                    continue

In [315]:
import re

for description in listes_prods_description[0].find_all("h3"):
    if description.find_next_sibling("p"):
        text = description.find_next_sibling("p").get_text()
        break

# Remove phrases starting with "Click for ..."
clean_text = re.sub(r"Click here for .*?Reviews", "", text, flags=re.DOTALL)
clean_text = re.sub(r"Click for .*?Reviews", "", clean_text, flags=re.DOTALL)
clean_text = re.sub(r"Check out .", "", clean_text, flags=re.DOTALL)
# Remove words with line breaks that don't have a period (".") at the end
clean_text = re.sub(r"(?m)^\w+(?: \w+)*\n(?!\w*\.)", "", clean_text)
clean_text = clean_text.replace('\n',"").split('. ')

In [316]:
csv_refrigerator_pro=CsvCustom("refrigerator_pro")
csv_refrigerator_pro.create_csv()
for index, (title, link) in enumerate(company_dict.items(), start=0):
    file=csv_refrigerator_pro.write_csv(title=title, link=link, description=clean_text[index], price="")
    
csv_refrigerator_pro.close_csv(file)  